In [1]:
import networkx as nx
import pandas as pd
import geopandas as gpd
import config as cfg
import numpy as np
import matplotlib.pyplot as plt

In [4]:
all_viajes = pd.read_csv('/Users/caro/Desktop/thesis_project/mobility_data/VIAJES/all_viajes_month_0322.csv')
income = gpd.read_file('/Users/caro/Desktop/thesis_project/segregation_indices/data/processed/geometries_and_income.geojson')
gdf = gpd.read_file(cfg.ZONIFICACION_DATA / 'distritos/madrid_gdf.geojson')  # Load your GeoJSON file into a GeoDataFrame
# gdf = gdf.to_crs(epsg=4326)
district_mapping = pd.read_csv('/Users/caro/Desktop/thesis_project/data_overview/outputs/districts_and_population.csv')

# Filter data
filtered_df = all_viajes.loc[(all_viajes['actividad_origen'] == 'casa')]
# home_to_work = filtered_df.loc[(filtered_df['actividad_destino'] == 'trabajo_estudio')] # only trips from home to work
# short_trips = all_viajes.loc[(all_viajes['distancia']=='0.5-2')]
# long_trips = all_viajes.loc[(all_viajes['distancia']=='10-50')]
# women = all_viajes.loc[(all_viajes['sexo'] == 'mujer')]
# high_income = all_viajes.loc[(all_viajes['renta']=='10-15')]

home_work = all_viajes[
    ((all_viajes['actividad_origen'] == 'casa') & (all_viajes['actividad_destino'] == 'trabajo_estudio')) |
    ((all_viajes['actividad_origen'] == 'trabajo_estudio') & (all_viajes['actividad_destino'] == 'casa'))
]


/Users/caro/Desktop/thesis_project/.thesis_env/lib/python3.11/site-packages/geopandas/array.py:334: UserWarning: Cannot set the CRS, falling back to None. The CRS support requires the 'pyproj' package, but it is not installed or does not import correctly. The functions depending on CRS will raise an error or may produce unexpected results.
  self.crs = crs
/Users/caro/Desktop/thesis_project/.thesis_env/lib/python3.11/site-packages/geopandas/geodataframe.py:59: UserWarning: Cannot set the CRS, falling back to None. The CRS support requires the 'pyproj' package, but it is not installed or does not import correctly. The functions depending on CRS will raise an error or may produce unexpected results.
  data.array.crs = crs
/Users/caro/Desktop/thesis_project/.thesis_env/lib/python3.11/site-packages/geopandas/geodataframe.py:407: UserWarning: Cannot set the CRS, falling back to None. The CRS support requires the 'pyproj' package, but it is not installed or does not import correctly. The fun

In [5]:
def build_trip_count(df, sociodemographic_var):
    trip_counts = df.groupby(['origen', 'destino', sociodemographic_var]).size().reset_index(name='trip_count')
    return trip_counts

def get_district_names(trip_counts):
    # get names of districts
    trip_counts = trip_counts.merge(district_mapping[['ID', 'name_2']], how='left', left_on='origen', right_on='ID')
    trip_counts = trip_counts.rename(columns={'name_2': 'origin'})
    trip_counts = trip_counts.merge(district_mapping[['ID', 'name_2']], how='left', left_on='destino', right_on='ID')
    trip_counts = trip_counts.rename(columns={'name_2': 'destination'})

    # drop extra columns
    trip_counts = trip_counts.drop(columns=['ID_x', 'ID_y'])

    return trip_counts

def get_income_data(trip_counts, income_var_1, income_var_2):
    # get origin incomes
    trip_counts = trip_counts.merge(
    income[['ID', income_var_1, income_var_2]], 
    left_on='origen', 
    right_on='ID', 
    how='left'
)
    # rename
    trip_counts.rename(columns={income_var_1: f'Origin {income_var_1}'}, inplace=True)
    trip_counts.rename(columns={income_var_2: f'Origin {income_var_2}'}, inplace=True)

    # drop extra columns
    trip_counts.drop(columns=['ID'], inplace=True)

    # get destination incomes
    trip_counts = trip_counts.merge(
        income[['ID', income_var_1, income_var_2]], 
        left_on='destino', 
        right_on='ID', 
        how='left'
    )

    # rename columns
    trip_counts.rename(columns={income_var_1: f'Destination {income_var_1}'}, inplace=True)
    trip_counts.rename(columns={income_var_2: f'Destination {income_var_2}'}, inplace=True)
    # drop extra columns
    trip_counts.drop(columns=['ID', 'origen', 'destino'], inplace=True)

    return trip_counts

def add_quantiles(trip_counts, income_var, n_quantiles=10):
    trip_counts[f'income decile origin {income_var}'] = pd.qcut(trip_counts[f'Origin {income_var}'], n_quantiles, labels=False, duplicates='drop')
    trip_counts[f'income decile destination {income_var}'] = pd.qcut(trip_counts[f'Destination {income_var}'], n_quantiles, labels=False, duplicates='drop') 
    return trip_counts


In [6]:
trip_counts = build_trip_count(home_work, 'renta')
trip_counts = get_district_names(trip_counts)
trip_counts = get_income_data(trip_counts, 'Gini Index', 'Median income per consumption unit')
trip_counts = add_quantiles(trip_counts, 'Median income per consumption unit')
trip_counts = add_quantiles(trip_counts, 'Gini Index')

In [13]:
trip_counts.sort_values(by='trip_count', ascending=False).tail(10)

,renta,trip_count,origin,destination,Origin Gini Index,Origin Median income per consumption unit,Destination Gini Index,Destination Median income per consumption unit,income decile origin Median income per consumption unit,income decile destination Median income per consumption unit,income decile origin Gini Index,income decile destination Gini Index
199,>15,338,Chamberí,Vicálvaro,37.9,29750,30.4,19250,7,2,7,0
328,>15,334,Carabanchel,Barajas,33.1,15750,33.2,25550,0,5,3,4
632,>15,332,Barajas,Carabanchel,33.2,25550,33.1,15750,5,0,4,3
565,>15,307,Vicálvaro,Chamberí,30.4,19250,37.9,29750,2,7,0,7
517,>15,299,Villaverde,Barajas,31.9,15050,33.2,25550,0,5,2,4
641,>15,295,Barajas,Villaverde,33.2,25550,31.9,15050,5,0,4,2
630,>15,273,Barajas,Latina,33.2,25550,31.8,17850,5,1,4,2
294,>15,272,Latina,Barajas,31.8,17850,33.2,25550,1,5,2,4
634,>15,193,Barajas,Usera,33.2,25550,33.4,14350,5,0,4,4
362,>15,171,Usera,Barajas,33.4,14350,33.2,25550,0,5,4,4


In [15]:
from scipy.stats import pearsonr, spearmanr

# Pearson correlation for linear relationships
pearson_corr_income, _ = pearsonr(trip_counts['Origin Median income per consumption unit'], trip_counts['trip_count'])
pearson_corr_gini, _ = pearsonr(trip_counts['Origin Gini Index'], trip_counts['trip_count'])

# Spearman correlation for non-linear relationships
spearman_corr_income, _ = spearmanr(trip_counts['Origin Median income per consumption unit'], trip_counts['trip_count'])
spearman_corr_gini, _ = spearmanr(trip_counts['Origin Gini Index'], trip_counts['trip_count'])

# Display the results
print(f"Pearson Correlation between Trip Count and Origin Median Income: {pearson_corr_income:.2f}")
print(f"Pearson Correlation between Trip Count and Origin Gini Index: {pearson_corr_gini:.2f}")
print(f"Spearman Correlation between Trip Count and Origin Median Income: {spearman_corr_income:.2f}")
print(f"Spearman Correlation between Trip Count and Origin Gini Index: {spearman_corr_gini:.2f}")


Pearson Correlation between Trip Count and Origin Median Income: 0.06
Pearson Correlation between Trip Count and Origin Gini Index: 0.10
Spearman Correlation between Trip Count and Origin Median Income: 0.11
Spearman Correlation between Trip Count and Origin Gini Index: 0.14


In [ ]:
# TODO: COMPLETE
def sort(trip_counts, var, threshold):
    high_rent_df = trip_counts[trip_counts[var] == '>15'].sort_values(by='trip_count', ascending=False)
    high_rent_top3 = high_rent_df.head(3)
    high_rent_bottom3 = high_rent_df.tail(3)

    # Filter medium rent (10-15)
    medium_rent_df = trip_counts[trip_counts[var] == '10-15'].sort_values(by='trip_count', ascending=False)
    medium_rent_top3 = medium_rent_df.head(3)
    medium_rent_bottom3 = medium_rent_df.tail(3)

    # Filter low rent (<10)
    low_rent_df = trip_counts[trip_counts[var] == '<10'].sort_values(by='trip_count', ascending=False)
    low_rent_top3 = low_rent_df.head(3)
    low_rent_bottom3 = low_rent_df.tail(3)

    # Combine the results into one DataFrame for easy viewing
    top3_bottom3_combined = pd.concat([
        high_rent_top3, high_rent_bottom3,
        medium_rent_top3, medium_rent_bottom3,
        low_rent_top3, low_rent_bottom3
    ])


In [48]:
high_rent_df = trip_counts[trip_counts['renta'] == '>15'].sort_values(by='trip_count', ascending=False)
high_rent_top3 = high_rent_df.head(3)
high_rent_bottom3 = high_rent_df.tail(3)

# Filter medium rent (10-15)
medium_rent_df = trip_counts[trip_counts['renta'] == '10-15'].sort_values(by='trip_count', ascending=False)
medium_rent_top3 = medium_rent_df.head(3)
medium_rent_bottom3 = medium_rent_df.tail(3)

# Filter low rent (<10)
low_rent_df = trip_counts[trip_counts['renta'] == '<10'].sort_values(by='trip_count', ascending=False)
low_rent_top3 = low_rent_df.head(3)
low_rent_bottom3 = low_rent_df.tail(3)

# Combine the results into one DataFrame for easy viewing
top3_bottom3_combined = pd.concat([
    high_rent_top3, high_rent_bottom3,
    medium_rent_top3, medium_rent_bottom3,
    low_rent_top3, low_rent_bottom3
])

In [49]:
high_rent_top3

,origen,destino,renta,trip_count,Origin median income per consumption unit,Origin Gini Index,Destination median income per consumption unit,Destination Gini Index,origen_name,destino_name
210,2807908,2807908,>15,23484,26950,34.4,26950,34.4,Fuencarral-El Pardo,Fuencarral-El Pardo
474,2807916,2807916,>15,21220,24850,37.2,24850,37.2,Hortaleza,Hortaleza
648,2807921,2807921,>15,17979,25550,33.2,25550,33.2,Barajas,Barajas


In [29]:
high_rent_bottom3

,origen,destino,renta,trip_count,origen_name,destino_name
294,2807910,2807921,>15,272,Latina,Barajas
634,2807921,2807912,>15,193,Barajas,Usera
362,2807912,2807921,>15,171,Usera,Barajas


In [33]:
medium_rent_top3

,origen,destino,renta,trip_count,origen_name,destino_name
547,2807918,2807918,10-15,20771,Villa de Vallecas,Villa de Vallecas
512,2807917,2807917,10-15,20738,Villaverde,Villaverde
617,2807920,2807920,10-15,19624,San Blas-Canillejas,San Blas-Canillejas


In [32]:
medium_rent_bottom3

,origen,destino,renta,trip_count,origen_name,destino_name
506,2807917,2807914,10-15,569,Villaverde,Moratalaz
406,2807914,2807910,10-15,515,Moratalaz,Latina
283,2807910,2807914,10-15,488,Latina,Moratalaz


In [25]:
trip_counts.sort_values(by='trip_count', ascending=False)

,origen,destino,renta,trip_count,origen_name,destino_name
210,2807908,2807908,>15,23484,Fuencarral-El Pardo,Fuencarral-El Pardo
474,2807916,2807916,>15,21220,Hortaleza,Hortaleza
547,2807918,2807918,10-15,20771,Villa de Vallecas,Villa de Vallecas
512,2807917,2807917,10-15,20738,Villaverde,Villaverde
617,2807920,2807920,10-15,19624,San Blas-Canillejas,San Blas-Canillejas
...,...,...,...,...,...,...
641,2807921,2807917,>15,295,Barajas,Villaverde
630,2807921,2807910,>15,273,Barajas,Latina
294,2807910,2807921,>15,272,Latina,Barajas
634,2807921,2807912,>15,193,Barajas,Usera


In [26]:
adj_1 = np.load('adjacency_matrices/filtered_df.npy')

In [27]:
adj_1

array([[0.62991398, 0.54832021, 0.42444451, 0.44823335, 0.23102356,
        0.19608283, 0.68091008, 0.24923646, 0.54181961, 0.40401804,
        0.36607918, 0.22057217, 0.13309423, 0.04578442, 0.13754938,
        0.13895038, 0.08691754, 0.08383536, 0.05220096, 0.11936451,
        0.10980975],
       [0.69567654, 0.58914511, 0.50867214, 0.23511446, 0.18437053,
        0.1485892 , 0.26422707, 0.217518  , 0.35559976, 0.35209728,
        0.57894589, 0.58017877, 0.32298467, 0.06273642, 0.12600521,
        0.15172742, 0.13469136, 0.10740004, 0.06385721, 0.11866401,
        0.0961921 ],
       [0.53030345, 0.43310264, 0.54142733, 0.57042786, 0.18678024,
        0.12967581, 0.2622937 , 0.18431449, 0.16192664, 0.07643812,
        0.11532965, 0.11165905, 0.56241419, 0.43876264, 0.23917734,
        0.12283897, 0.05982235, 0.10521449, 0.0791    , 0.10431786,
        0.09296982],
       [0.39527586, 0.12177422, 0.45563059, 0.56104122, 0.57258539,
        0.19064698, 0.52172938, 0.22023593, 0.1902266